<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-and-read-package-and-file,-preliminary-check-up-on-dataframe" data-toc-modified-id="Load-and-read-package-and-file,-preliminary-check-up-on-dataframe-1">Load and read package and file, preliminary check up on dataframe</a></span></li><li><span><a href="#Task1:-Cleaning-Data" data-toc-modified-id="Task1:-Cleaning-Data-2">Task1: Cleaning Data</a></span></li></ul></div>

# Load and read package and file, preliminary check up on dataframe

* Load three main CSV file: customers, products, transactions and explore what are information included in each table by using .head()

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
customers=pd.read_csv('customers.csv')
products=pd.read_csv('products.csv')
transactions=pd.read_csv('transactions.csv')



In [3]:
customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [4]:
products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


* investigate wrong value in products dataframe

In [5]:
products[products.price <0]

,id_prod,price,categ
731,T_0,-1.0,0


* check out transaction dataframe with id_prod T_0

In [6]:
transactions[transactions.id_prod == 'T_0']

## It seems like there is also weird format ct_1 and ct_0 in client_id column so this will need cleaning too

,id_prod,date,session_id,client_id
1431,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1
2365,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_1
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1
5955,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0
7283,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_1
...,...,...,...,...
332594,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0
332705,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1
332730,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_1
333442,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1


In [7]:
customers.loc[(customers['client_id'] == 'ct_1')&(customers['client_id'] == 'ct_0')]

,client_id,sex,birth


# Task1: Cleaning Data 


Cleaning customers datafrme: correct data type, add age column for future analysis, get rid of testing customer ids

In [8]:
customers['year']=int('2021')
customers['age']=customers['year']-customers['birth']
customers=customers.loc[(customers['client_id'] != 'ct_1')&(customers['client_id'] != 'ct_0')]
customers.head()

,client_id,sex,birth,year,age
0,c_4410,f,1967,2021,54
1,c_7839,f,1975,2021,46
2,c_1699,f,1984,2021,37
3,c_5961,f,1962,2021,59
4,c_5320,m,1943,2021,78


* Use decribe to check if there are outliers or any missing/odd values

In [9]:
# check out data if there are any missing/odd values, outliers

customers.describe()

,birth,year,age
count,8621.000000,8621.0,8621.000000
mean,1978.275606,2021.0,42.724394
std,16.917958,0.0,16.917958
min,1929.000000,2021.0,17.000000
25%,1966.000000,2021.0,29.000000
50%,1979.000000,2021.0,42.000000
75%,1992.000000,2021.0,55.000000
max,2004.000000,2021.0,92.000000


In [10]:
#check data type of each column, to see if there are any outliers or data with wrong formatting

print(customers['sex'].dtypes)
print(customers['client_id'].dtypes)
print(customers['birth'].dtypes)
print(customers['year'].dtypes)
print(customers['age'].dtypes)

object
object
int64
int64
int64


Cleaning transactions dataframe: get rid of outliers on id_prod,session_id,client_id,date columns. Also change formate of date column into datetime and add new column month/day, for future analysis

In [11]:
transactions=transactions.loc[(transactions['id_prod'] != "T_0")]
transactions=transactions.loc[(transactions['session_id'] != "s_0")]
transactions=transactions.loc[(transactions['client_id'] != 'ct_1')&(transactions['client_id'] != 'ct_0')]
boolean_finding= transactions['date'].str.contains('test_').any()
boolean_finding
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d %H:%M:%S.%f',errors='coerce')
transactions['month'] = transactions['date'].dt.month
transactions['day'] = transactions['date'].dt.day
transactions.dropna(subset = ["month"], inplace=True)
transactions.head()



,id_prod,date,session_id,client_id,month,day
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4,10
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,2,3
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,9,23
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,10,17
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242,7,17


* Use decribe to check if there are outliers or any missing/odd values

In [12]:
# check description of each column to find if there are any outliers

transactions.describe()

,month,day
count,336816.000000,336816.000000
mean,6.482133,15.801628
std,3.519807,8.818357
min,1.000000,1.000000
25%,3.000000,8.000000
50%,6.000000,16.000000
75%,9.000000,23.000000
max,12.000000,31.000000


In [13]:
#check data type of each column, to see if there are any outliers

print(transactions['id_prod'].dtypes)
print(transactions['date'].dtypes)
print(transactions['session_id'].dtypes)
print(transactions['client_id'].dtypes)
print(transactions['month'].dtypes)

object
datetime64[ns]
object
object
int64


* Use decribe to check if there are outliers or any missing/odd values

In [14]:
products.describe()

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


In [15]:
#check data type of each column, to see if there are any outliers

print(products['price'].dtypes)
print(products['categ'].dtypes)


float64
int64


Lastly, export clean version of CSV file 

 * check if there are any duplicated rows and null in data frame

In [16]:
products.isnull().mean()

id_prod    0.0
price      0.0
categ      0.0
dtype: float64

In [17]:
customers.isnull().mean()

client_id    0.0
sex          0.0
birth        0.0
year         0.0
age          0.0
dtype: float64

In [18]:
transactions.isnull().mean()

id_prod       0.0
date          0.0
session_id    0.0
client_id     0.0
month         0.0
day           0.0
dtype: float64

In [19]:
print(customers.duplicated(subset=['client_id']).any())


False


In [20]:
print(products.duplicated(subset=['id_prod']).any())

False


In [21]:
print(transactions.duplicated(subset=['session_id']).any())
#transactions.drop_duplicates(subset ="session_id", keep = False, inplace = True)
#for session id, its normal to have duplicate as it is possible that customer might have bought multiple items at one session.

True


* It seems transacionts data frame still hae duplicate in terms of session id, next code will remove it and we will check again

Lastly, export clean version of CSV file

In [22]:
products.to_csv('products_cleaned.csv', index=False)  
customers.to_csv('customers_cleaned.csv', index=False)  
transactions.to_csv('transactions_cleaned.csv', index=False)  